In [189]:
import pandas as pd
from datetime import datetime
import numpy as np

# Data Preprocessing

### Removing useless columns and identifying columns to encode

In [190]:
data = pd.read_csv(r'./data/history.csv', low_memory=False)
columns_to_remove = ['id', 'country_code', 'currency_code', 'net_of_fees', 'cashflows_custody_fee', 'fee_paid_separately', 'custody_fee_withdrawal',
                     'is_fee_exempt', 'include_client_consolidation', 'credit_limit_type', 'sss_type', 'sss_agent', 'is_midwest_clearing_account',
                     'terminal_code', 'target_grantor_grantee_flag', 'iso_funds_code', 'esir_number', 'shareholder_language', 'shareholder_language',
                     'conjunction', 'function_code', 'tms_settlement_location', 'portfolio_cost_method', 'portfolio_name_address_option',
                     'portfolio_report_option', 'portfolio_summary_option', 'interactive_portfolio_code', 'deceased_fair_market_value', 'rep_commission_rate',
                     'dup_trip_quad_code', 'special_fee_code', 'non_calendar_year_end', 'resp_specimen_plan', 'sss_location', 'last_maintenance_user', 'last_maintenance_time',
                     'retail_last_maintenance_user', 'retail_last_maintenance_time', 'arp_pension_origin']

cols_to_numerify_from_bool = ["is_registered", "is_active",
    "use_client_address", "is_spousal", "is_arp_locked",
    "use_hand_delivery", "use_mail", "share_name_address_to_issuer",
    "shareholder_instructions_received", "rrsp_limit_reached", "is_portfolio_account",
    "has_no_min_commission", "is_tms_eligible", "is_agent_bbs_participant",
    "is_parameters_account", "is_spousal_transfer", "spousal_age_flag", "has_multiple_name",
    "discretionary_trading_authorized", "receive_general_mailings", "has_discrete_auth",
    "is_non_objecting_beneficial_owner", "is_objecting_to_disclose_info", "consent_to_pay_for_mail",
    "consent_to_email_delivery", "has_received_instruction", "is_broker_account",
    "is_inventory_account", "is_gl_account", "is_control_account", "is_extract_eligible",
    "is_pledged", "is_resp", "use_original_date_for_payment_calc", "is_family_resp",
    "is_hrdc_resp", "is_plan_grandfathered", "is_olob", "visible_in_reports", 'inserted_at', 'updated_at']

cols_to_encode = ['class_id', 'type_code', 'debit_code', 'contract_type',
                  'branch', 'retail_plan',
                  'special_tag', 'guarantee_gtor_type', 'dividend_confirm_code', 'options_trading_type',
                  'interest_dividend_conversion_type']

cols_null_to_numeric = ['rep_commission_override', 'loan_limit_override', 'non_plan_book_value_flag']

cols_date_to_numeric = ['last_trade_date', 'inception_date', 'last_update_date', 'plan_effective_date',
                        'plan_end_date', 'rrif_original_date']

special_cols = ['language_code', 'title', 'risk_tolerance', 'investment_objective', 'mailing_consent', 'number_of_beneficiaries', 'label']

# drop useless columns
data = data.drop(columns=columns_to_remove)

# store for check later
data_columns = data.columns

data.columns


Index(['type_code', 'is_registered', 'is_active', 'class_id', 'debit_code',
       'last_trade_date', 'contract_type', 'inception_date', 'branch',
       'use_client_address', 'retail_plan', 'is_spousal', 'is_arp_locked',
       'language_code', 'dividend_confirm_code', 'options_trading_type',
       'rep_commission_override', 'interest_dividend_conversion_type',
       'guarantee_gtor_type', 'use_hand_delivery', 'use_mail',
       'share_name_address_to_issuer', 'shareholder_instructions_received',
       'rrsp_limit_reached', 'is_portfolio_account', 'has_no_min_commission',
       'is_tms_eligible', 'is_agent_bbs_participant', 'is_parameters_account',
       'is_spousal_transfer', 'spousal_age_flag', 'has_multiple_name',
       'discretionary_trading_authorized', 'special_tag', 'title',
       'receive_general_mailings', 'loan_limit_override',
       'non_plan_book_value_flag', 'has_discrete_auth', 'last_update_date',
       'is_non_objecting_beneficial_owner', 'is_objecting_to_discl

In [5]:
# get an overview of what the dataset looks like
data.head()

,type_code,is_registered,is_active,class_id,debit_code,last_trade_date,contract_type,inception_date,branch,use_client_address,...,rrif_original_date,use_original_date_for_payment_calc,is_family_resp,is_hrdc_resp,is_plan_grandfathered,inserted_at,updated_at,is_olob,visible_in_reports,label
0,CASH SWEEP,f,t,3.0,0,NaN,NaN,2017-08-18,IAVM,f,...,NaN,NaN,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,f,Churn
1,RRIF,t,t,5.0,C,2022-07-04,18.0,2018-11-08,IAVM,t,...,2018-11-08,f,f,f,f,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,t,Churn
2,SPOUSAL RRSP,t,t,5.0,A,2018-07-05,16.0,2009-04-08,IAVM,f,...,2009-04-08,f,f,f,f,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,f,Churn
3,CASH,f,t,3.0,T,NaN,12.0,2021-01-04,IAVM,t,...,NaN,NaN,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,t,Churn
4,CASH,f,t,3.0,T,NaN,17.0,2017-07-21,IAVM,f,...,NaN,NaN,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 20:52:08.352646+00,f,f,Churn


### REGULAR PREPROCESSING (BOOL TO INT, ONE HOT ENCODING, NULL TO INT, DATE TO INT)

In [191]:
# bool to numeric function (cols_to_numerify_from_bool)
def bool_to_numeric(df, feature):
    bool_to_numeric_dict = {"t": 1, "f": 0}
    df[feature] = df[feature].map(bool_to_numeric_dict).fillna(-1)
    return df

for col in cols_to_numerify_from_bool:
  try:
    data = bool_to_numeric(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

data[cols_to_numerify_from_bool].head()

,is_registered,is_active,use_client_address,is_spousal,is_arp_locked,use_hand_delivery,use_mail,share_name_address_to_issuer,shareholder_instructions_received,rrsp_limit_reached,...,is_pledged,is_resp,use_original_date_for_payment_calc,is_family_resp,is_hrdc_resp,is_plan_grandfathered,is_olob,visible_in_reports,inserted_at,updated_at
0,0,1,0,-1.0,-1.0,0,0,1,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,-1.0,-1.0
1,1,1,1,0.0,0.0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,-1.0,-1.0
2,1,1,0,1.0,0.0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,-1.0,-1.0
3,0,1,1,-1.0,-1.0,0,0,1,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,1,-1.0,-1.0
4,0,1,0,-1.0,-1.0,0,0,1,0,0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,-1.0,-1.0


In [192]:
# one hot encode function (cols_to_encode)
def encode_and_bind(original_dataframe, feature_to_encode):
    temp = pd.get_dummies(original_dataframe[[feature_to_encode]])
    df = pd.concat([original_dataframe, temp], axis=1)
    df = df.drop([feature_to_encode], axis=1)
    return(df)

for col in cols_to_encode:
  try:
    data = encode_and_bind(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

data.head()

,is_registered,is_active,last_trade_date,inception_date,use_client_address,is_spousal,is_arp_locked,language_code,rep_commission_override,use_hand_delivery,...,retail_plan_TF,special_tag_CURATELLE,special_tag_ESTATE OF,special_tag_FIDUCIE,special_tag_JTIC,special_tag_JTWROS,special_tag_NOT FOUND,special_tag_SUCCESSION,special_tag_TRUST,special_tag_TUTELLE
0,0,1,NaN,2017-08-18,0,-1.0,-1.0,E,0.0,0,...,False,False,False,False,False,False,False,False,False,False
1,1,1,2022-07-04,2018-11-08,1,0.0,0.0,E,0.0,0,...,False,False,False,False,False,False,False,False,False,False
2,1,1,2018-07-05,2009-04-08,0,1.0,0.0,F,0.0,0,...,False,False,False,False,False,False,False,False,False,False
3,0,1,NaN,2021-01-04,1,-1.0,-1.0,E,0.0,0,...,False,False,False,False,False,False,False,False,False,False
4,0,1,NaN,2017-07-21,0,-1.0,-1.0,E,0.0,0,...,False,False,False,False,False,False,False,False,False,False


In [193]:
# for these, null = 0, has value = 1 (cols_null_to_numeric)
def null_to_numeric(df, feature):
  df[feature] = df[feature].notnull().astype(int)
  return df

for col in cols_null_to_numeric:
  try:
    data = null_to_numeric(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

data[cols_null_to_numeric].head()

,rep_commission_override,loan_limit_override,non_plan_book_value_flag
0,1,0,1
1,1,0,0
2,1,0,0
3,1,0,1
4,1,0,1


In [194]:
# modifying (cols_date_to_numeric)
date_format = "%Y-%m-%d"

# Assuming `data` is your DataFrame and `date_columns` is a list of date columns
for column in cols_date_to_numeric:
    try:
        # Convert to datetime
        data[column] = pd.to_datetime(data[column], errors='coerce')

        # Calculate the difference in days from today
        data[column] = (datetime.now() - data[column]).dt.days

    except Exception as e:
        print(f'Error: {e}')
        continue

# Calculate the average of non-null values for each column
column_averages = {column: data[column].dropna().mean() for column in cols_date_to_numeric}

# Replace null values with the average of non-null values
for column in cols_date_to_numeric:
    data[column] = data[column].fillna(column_averages[column])

data[cols_date_to_numeric].head()

,last_trade_date,inception_date,last_update_date,plan_effective_date,plan_end_date,rrif_original_date
0,1395.999573,2380.0,1889.917826,2333.864163,1706.417913,2490.012951
1,599.000000,1933.0,533.000000,1933.000000,1706.417913,1933.000000
2,2059.000000,5434.0,3593.000000,5434.000000,2047.000000,5434.000000
3,1395.999573,1145.0,1145.000000,2333.864163,1706.417913,2490.012951
4,1395.999573,2408.0,2405.000000,2333.864163,1706.417913,2490.012951


### SPECIAL COLUMN PREPROCESSING

In [195]:
# SPECIAL COLUMN: language_code
# english = 0 , french = 1
def update_language(df, feature):
  language_to_numeric_dict = {"F": 1, "E": 0}
  df[feature] = df[feature].map(language_to_numeric_dict)
  return df

data = update_language(data, 'language_code')

data['language_code'].head()

0    0.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: language_code, dtype: float64

In [196]:
# SPECIAL COLUMN: title
def gender_to_number(df, feature):
    mapping = {
        'male': ['mr', 'monsieur', 'sir', 'mister'],
        'female': ['miss', 'ms', 'mrs', 'madame', 'mme', 'mlle', 'mademoiselle', 'mm']
    }
    for key in mapping.keys():
        mapping[key] = [phrase.lower() for phrase in mapping[key]]

    def map_gender(value):
        try:
            value_lower = str(value).lower()
            if pd.isna(value) or value == "":
                return "none_provided"
            elif any(phrase in value_lower for phrase in mapping['male']):
                return "male"
            elif any(phrase in value_lower for phrase in mapping['female']):
                return "female"
            else:
                return "other"
        except AttributeError:
            return "none_provided"
    df[feature] = df[feature].apply(map_gender)
    df = encode_and_bind(df, feature)
    return df

data = gender_to_number(data, 'title')

In [198]:
# SPECIAL COLUMN: risk_tolerance, investment_objective
def convert_risk_to_numeric(df, feature):

    unique_risks = set()

    for idx, row in df.iterrows():
        risk_str = row[feature]
        risk_str = str(risk_str)
        if pd.notna(risk_str):
            for char in risk_str:
                if char.isalpha():
                    unique_risks.add(char)

    col_names = [f'{feature}_{risk}' for risk in unique_risks]

    for col_name in col_names:
        df[col_name] = 0

    df[f'{feature}_exists'] = 1

    for idx, row in df.iterrows():
        risk_str = row[feature]
        risk_str = str(risk_str)
        if pd.notna(risk_str):
            for i in range(len(risk_str)):
                if risk_str[i].isalpha():
                    col_name = f'{feature}_{risk_str[i]}'
                    j = i + 1
                    while j < len(risk_str) and risk_str[j].isdigit():
                        j += 1
                    try:
                        value = int(risk_str[i + 1:j]) if j > i + 1 else 0
                    except (ValueError, IndexError):
                        value = 0
                        df.at[idx, f'{feature}_exists'] = 0
                    df.at[idx, col_name] = value

    # mean_values = df[df[f'{feature}_exists'] == 1][col_names].mean()
    # for idx, row in df.iterrows():
    #     if all(row[col] == 0 for col in col_names):
    #         df.loc[idx, col_names] = mean_values
    #         df.at[idx, f'{feature}_exists'] = 0

    df = df.fillna(0)

    df.drop(columns=[feature], inplace=True)

    return df

data = convert_risk_to_numeric(data, 'risk_tolerance')
data = convert_risk_to_numeric(data, 'investment_objective')

data.head()

,is_registered,is_active,last_trade_date,inception_date,use_client_address,is_spousal,is_arp_locked,language_code,rep_commission_override,use_hand_delivery,...,investment_objective_E,investment_objective_A,investment_objective_G,investment_objective_I,investment_objective_R,investment_objective_S,investment_objective_M,investment_objective_B,investment_objective_X,investment_objective_exists
0,0,1,1395.999573,2380.0,0,-1.0,-1.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,599.000000,1933.0,1,0.0,0.0,0.0,1,0,...,0,0,85,0,0,10,0,5,0,1
2,1,1,2059.000000,5434.0,0,1.0,0.0,1.0,1,0,...,0,0,50,0,0,50,0,0,0,1
3,0,1,1395.999573,1145.0,1,-1.0,-1.0,0.0,1,0,...,0,0,99,0,0,0,0,0,0,1
4,0,1,1395.999573,2408.0,0,-1.0,-1.0,0.0,1,0,...,0,0,99,0,0,0,0,0,0,1


In [199]:
# SPECIAL COLUMNS: mailing_consent, number_of_beneficiaries, label
data['mailing_consent'] = data['mailing_consent'].fillna(0)
data['number_of_beneficiaries'] = data['number_of_beneficiaries'].fillna(0)
data['label'] = data['label'].map({'Churn': 1, 'No Churn': 0})

# fill all other null cells with -1
data = data.fillna(-1)

### THE FINAL DATASET

In [ ]:
data.columns

Index(['is_registered', 'is_active', 'last_trade_date', 'inception_date',
       'use_client_address', 'is_spousal', 'is_arp_locked', 'language_code',
       'rep_commission_override', 'use_hand_delivery',
       ...
       'investment_objective_E', 'investment_objective_A',
       'investment_objective_G', 'investment_objective_I',
       'investment_objective_R', 'investment_objective_S',
       'investment_objective_M', 'investment_objective_B',
       'investment_objective_X', 'investment_objective_exists'],
      dtype='object', length=158)

In [ ]:
# THE FINAL DATASET
data.head()

,is_registered,is_active,last_trade_date,inception_date,use_client_address,is_spousal,is_arp_locked,language_code,rep_commission_override,use_hand_delivery,...,investment_objective_E,investment_objective_A,investment_objective_G,investment_objective_I,investment_objective_R,investment_objective_S,investment_objective_M,investment_objective_B,investment_objective_X,investment_objective_exists
0,0,1,1395.999573,2380.0,0,-1.0,-1.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,599.000000,1933.0,1,0.0,0.0,0.0,1,0,...,0,0,85,0,0,10,0,5,0,1
2,1,1,2059.000000,5434.0,0,1.0,0.0,1.0,1,0,...,0,0,50,0,0,50,0,0,0,1
3,0,1,1395.999573,1145.0,1,-1.0,-1.0,0.0,1,0,...,0,0,99,0,0,0,0,0,0,1
4,0,1,1395.999573,2408.0,0,-1.0,-1.0,0.0,1,0,...,0,0,99,0,0,0,0,0,0,1


# Training

### Implementing Partial Least Squares and Minimum Redundancy Maximum Relevance (MRMR) for dimensionality reduction

In [200]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.feature_selection import mutual_info_regression
from xgboost import XGBClassifier

NUMBER_OF_FEATURES_PLS = 50
NUMBER_OF_FEATURES_MRMR = 30

train_f1 = []
test_f1 = []

# SPLITTING UP THE DATA INTO INPUT AND OUTPUT
X = data.drop(columns=['label'])
y = data['label']

# SCALE THE DATA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# IMPLEMENT PLS REGRESSION FOR DIMENSIONALITY REDUCTION
pls = PLSRegression(n_components=NUMBER_OF_FEATURES_PLS)
pls.fit(X_scaled, y)
X_pls = pls.transform(X_scaled)

# IMPLEMENTING MRMR
# Feature Relevance
mi_scores = mutual_info_regression(X_pls, y)

# Feature Redundancy
cos_sim = np.abs(np.corrcoef(X_pls.T))

# Calculate MRMR
mrmr_scores = mi_scores / np.maximum(np.max(mi_scores) - np.mean(mi_scores), 1e-10)
mrmr_scores -= np.mean(cos_sim, axis=1)

# Select Features using MRMR
selected_features = np.argsort(mrmr_scores)[::-1][:NUMBER_OF_FEATURES_MRMR]


selected_feature_names = X.columns[selected_features]
data = data[selected_feature_names]
data['label'] = y
data.head()

,is_registered,is_active,last_trade_date,inception_date,use_client_address,use_mail,share_name_address_to_issuer,has_received_instruction,is_portfolio_account,discretionary_trading_authorized,...,is_tms_eligible,is_inventory_account,use_hand_delivery,number_of_beneficiaries,is_parameters_account,is_arp_locked,is_hrdc_resp,rep_commission_override,is_resp,label
0,0,1,1395.999573,2380.0,0,0,1,0,0,0.0,...,0,0,0,0.0,0,-1.0,-1.0,1,-1.0,1
1,1,1,599.000000,1933.0,1,0,1,1,0,0.0,...,0,0,0,0.0,0,0.0,0.0,1,0.0,1
2,1,1,2059.000000,5434.0,0,0,1,1,0,0.0,...,0,0,0,0.0,0,0.0,0.0,1,0.0,1
3,0,1,1395.999573,1145.0,1,0,1,1,0,0.0,...,0,0,0,0.0,0,-1.0,-1.0,1,-1.0,1
4,0,1,1395.999573,2408.0,0,0,1,1,0,0.0,...,0,0,0,0.0,0,-1.0,-1.0,1,-1.0,1


In [ ]:
data.columns

Index(['is_registered', 'is_active', 'last_trade_date', 'inception_date',
       'use_client_address', 'share_name_address_to_issuer', 'use_mail',
       'has_received_instruction', 'is_portfolio_account',
       'discretionary_trading_authorized', 'shareholder_instructions_received',
       'is_spousal', 'inserted_at', 'is_broker_account', 'is_extract_eligible',
       'has_no_min_commission', 'is_pledged', 'is_objecting_to_disclose_info',
       'is_agent_bbs_participant', 'is_tms_eligible', 'rrsp_limit_reached',
       'consent_to_pay_for_mail', 'number_of_beneficiaries',
       'use_hand_delivery', 'is_inventory_account', 'is_parameters_account',
       'plan_effective_date', 'mailing_consent', 'rep_commission_override',
       'is_hrdc_resp', 'label', 'segment'],
      dtype='object')

### K-means clustering of data



In [201]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

n_clusters = 20  # Choose the number of clusters (segments)
X = data.drop(columns=['label', 'segment'])  # Features

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=n_clusters, random_state=1)
data['segment'] = kmeans.fit_predict(X_scaled)


KeyError: "['segment'] not found in axis"

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min

cluster_counts = data.groupby('segment').size()
pure_clusters = []
for segment, count in cluster_counts.items():
    label_count = data[(data['label'] == 1) & (data['segment'] == segment)].shape[0]
    if label_count == count:
        pure_clusters.append(segment)

# Find nearest non-pure cluster for each pure cluster
nearest_non_pure_clusters = {}
for pure_cluster in pure_clusters:
    non_pure_cluster_indices, non_pure_cluster_distances = pairwise_distances_argmin_min(
        X_scaled[kmeans.labels_ != pure_cluster], 
        [kmeans.cluster_centers_[pure_cluster]]
    )
    nearest_non_pure_cluster_index = non_pure_cluster_indices[0]
    nearest_non_pure_clusters[pure_cluster] = kmeans.labels_[kmeans.labels_ != pure_cluster][nearest_non_pure_cluster_index]

# Reassign cluster values for datapoints in pure clusters
for pure_cluster, nearest_non_pure_cluster in nearest_non_pure_clusters.items():
    data.loc[data['segment'] == pure_cluster, 'segment'] = nearest_non_pure_cluster


### Implementing StratifiedKFold on each grouping from K-means, and training a XGBoosted model for each grouping of data

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score


segment_models = {}
SPLITS = 5

skf = StratifiedKFold(n_splits=SPLITS, shuffle=True, random_state=1)

print(n_clusters)

for segment_id in range(n_clusters):
    segment_data = data[data['segment'] == segment_id]

    if len(segment_data) == 0:
        continue  # Skip empty segments
    

    # print(segment_data['segment'])
    X_segment = segment_data.drop(columns=['label', 'segment'])
    y_segment = segment_data['label']

    # Scale X_segment
    scaler = StandardScaler()
    X_segment = scaler.fit_transform(X_segment)

    # MODEL
    xgb_model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1)
    segment_metrics = {'accuracy': []}  # Dictionary to store performance metrics


    # IF THERE IS ONLY ONE CLASS IN THE SEGMENT THEN SKIP TRAINING
    unique_values = np.unique(y_segment)
    if len(unique_values) == 1:
        default_model = "churn" if unique_values[0] == 1 else "not churn"
        segment_models[segment_id] = {'model': default_model, 'metrics': segment_metrics}
        print(segment_models[segment_id])
        continue


    for train_index, test_index in skf.split(X_segment, y_segment):
        X_train, X_test = X_segment[train_index], X_segment[test_index]
        y_train, y_test = y_segment.iloc[train_index], y_segment.iloc[test_index]

        unique_values = np.unique(y_train)


        xgb_model.fit(X_train, y_train)

        y_pred = xgb_model.predict(X_test)
        accuracy = f1_score(y_test, y_pred)
        segment_metrics['accuracy'].append(accuracy)


    segment_models[segment_id] = {'model': xgb_model, 'metrics': segment_metrics}
    print(segment_models[segment_id])

20


c:\Users\yunfe\anaconda3\envs\tensorflowenv\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


{'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...), 'metrics': {'accuracy': [1.0, 0.9999108813831209, 0.9999108813831209, 0.9999108813831209, 1.0]}}
{'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
       

c:\Users\yunfe\anaconda3\envs\tensorflowenv\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


{'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...), 'metrics': {'accuracy': [0.8603435399551904, 0.8613569321533924, 0.8670095518001469, 0.865826538176427, 0.8707280832095097]}}


# Testing and Validation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances

# Split the data into 80% training and 20% testing
X_train, X_val, y_train, y_val = train_test_split(data.drop(['label', 'segment'], axis=1), data['label'], test_size=0.2, random_state=1)

scaler = StandardScaler()
X_val = scaler.fit_transform(X_val)

def predict_churn_probabilities(new_data):
    # Get unique segment IDs present in the data
    unique_segments = data['segment'].unique()
    
    # Filter centroids to include only those corresponding to the segments present in the data
    centroids = kmeans.cluster_centers_[unique_segments]
    
    # Calculate distances to filtered centroids
    distances = pairwise_distances(new_data, centroids)

    # Calculate probability of being in each cluster
    cluster_probabilities = 1 / distances
    cluster_probabilities /= np.sum(cluster_probabilities, axis=1, keepdims=True)

    churn_probabilities = []
    for i, row in enumerate(new_data):
        cluster_id_probs = cluster_probabilities[i]
        weighted_churn_prob = 0

        for j, segment_id in enumerate(unique_segments):
            xgb_model = segment_models[segment_id]['model']
            if isinstance(xgb_model, str):  # Default model (e.g., "churn" or "not churn")
                churn_prob = 1 if xgb_model == "churn" else 0
            else:  # XGBoost model
                if xgb_model != "churn":  # Exclude "churn" indices
                    churn_prob = xgb_model.predict_proba(row.reshape(1, -1))[0, 1]
                else:
                    churn_prob = 0

            weighted_churn_prob += churn_prob * cluster_id_probs[j]

        churn_probabilities.append(weighted_churn_prob)

    return np.round(churn_probabilities)

y_pred = predict_churn_probabilities(X_val)


In [158]:
from sklearn.base import BaseEstimator, ClassifierMixin

class ChurnPredictor(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        # No training needed, as the model is already trained
        return self

    def predict(self, X):
        return predict_churn_probabilities(X)

estimator = ChurnPredictor(model=predict_churn_probabilities)

In [159]:
from sklearn.model_selection import train_test_split
y_pred_rounded = np.round(y_pred)


f1 = f1_score(y_val, y_pred_rounded)

print(f"F1 Score: {f1}")

F1 Score: 0.957619501815914


In [160]:
from sklearn.model_selection import cross_val_score

scaler = StandardScaler()
input = data.drop(['label', 'segment'], axis=1)
input = scaler.fit_transform(input)

cv_scores = cross_val_score(estimator=estimator, X=input, y=data['label'].to_numpy(), cv=5)
print(f"Average CV score: {np.mean(cv_scores)}")

Average CV score: 0.9318188903325304


# Predicting test results

In [178]:
data = pd.read_csv("./data/test.csv", low_memory=False)

for col in cols_to_numerify_from_bool:
  try:
    data = bool_to_numeric(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

for col in cols_to_encode:
  try:
    data = encode_and_bind(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

for col in cols_null_to_numeric:
  try:
    data = null_to_numeric(data, col)
  except Exception as e:
    print(f'Error: {e}')
    continue

# Assuming `data` is your DataFrame and `date_columns` is a list of date columns
for column in cols_date_to_numeric:
    try:
        # Convert to datetime
        data[column] = pd.to_datetime(data[column], errors='coerce')

        # Calculate the difference in days from today
        data[column] = (datetime.now() - data[column]).dt.days

    except Exception as e:
        print(f'Error: {e}')
        continue

# Calculate the average of non-null values for each column
column_averages = {column: data[column].dropna().mean() for column in cols_date_to_numeric}

# Replace null values with the average of non-null values
for column in cols_date_to_numeric:
    data[column] = data[column].fillna(column_averages[column])

# SPECIAL COLUMN: language_code
# english = 0 , french = 1
data = update_language(data, 'language_code')


# SPECIAL COLUMN: title
data = gender_to_number(data, 'title')

data = convert_risk_to_numeric(data, 'risk_tolerance')
data = convert_risk_to_numeric(data, 'investment_objective')


# SPECIAL COLUMNS: mailing_consent, number_of_beneficiaries
data['mailing_consent'] = data['mailing_consent'].fillna(0)
data['number_of_beneficiaries'] = data['number_of_beneficiaries'].fillna(0)

# fill all other null cells with -1
data = data.fillna(-1)
id = data['id']

### Show the data

In [180]:
data = data[selected_feature_names]
data.head()

,is_registered,is_active,last_trade_date,inception_date,use_client_address,share_name_address_to_issuer,use_mail,has_received_instruction,is_portfolio_account,discretionary_trading_authorized,...,rrsp_limit_reached,consent_to_pay_for_mail,number_of_beneficiaries,use_hand_delivery,is_inventory_account,is_parameters_account,plan_effective_date,mailing_consent,rep_commission_override,is_hrdc_resp
0,0,1,1399.722133,2408.0,1,1,0,1,0,0.0,...,0,0,0.0,0,0,0,2340.707031,0.0,1,-1.0
1,1,1,1205.000000,1446.0,1,1,0,1,0,0.0,...,0,0,0.0,0,0,0,1446.000000,0.0,1,0.0
2,0,1,1399.722133,409.0,1,1,0,1,0,0.0,...,0,0,0.0,0,0,0,2340.707031,0.0,1,-1.0
3,1,1,1452.000000,2408.0,0,1,0,1,0,0.0,...,0,0,0.0,0,0,0,2408.000000,0.0,1,0.0
4,0,1,4711.000000,5483.0,1,1,0,1,0,-1.0,...,0,0,0.0,0,0,0,2340.707031,0.0,1,-1.0


### Performing K-means on the test data

In [187]:


scaled_test_data = scaler.transform(data)
test_data['segment'] = kmeans.predict(scaled_test_data)
scaled_x = scaler.fit_transform(data)

kmeans = KMeans(n_clusters=n_clusters, random_state=1)
data['segment'] = kmeans.fit_predict(scaled_x)

In [204]:
def predict(new_data):
    # Get unique segment IDs present in the data
    unique_segments = data['segment'].unique()

    centroids = kmeans.cluster_centers_
    
    # Calculate distances to filtered centroids
    distances = pairwise_distances(new_data, centroids)

    # Calculate probability of being in each cluster
    cluster_probabilities = 1 / distances
    cluster_probabilities /= np.sum(cluster_probabilities, axis=1, keepdims=True)

    churn_probabilities = []
    for i, row in enumerate(new_data):
        cluster_id_probs = cluster_probabilities[i]
        weighted_churn_prob = 0

        for j, segment_id in enumerate(unique_segments):
            xgb_model = segment_models[segment_id]['model']
            if isinstance(xgb_model, str):  # Default model (e.g., "churn" or "not churn")
                churn_prob = 1 if xgb_model == "churn" else 0
            else:  # XGBoost model
                if xgb_model != "churn":  # Exclude "churn" indices
                    churn_prob = xgb_model.predict_proba(row.reshape(1, -1))[0, 1]
                else:
                    churn_prob = 0

            weighted_churn_prob += churn_prob * cluster_id_probs[j]

        churn_probabilities.append(weighted_churn_prob)

    return np.round(churn_probabilities)


pred = predict(data)

KeyError: 'segment'